# Linear Integer Programming

The following linear program and related code come from an [example from Wikipedia](https://en.wikipedia.org/wiki/Integer_programming#Example) that is solved via an [example in the SciPy documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.milp.html).

## Example

Maximize $y$ given:

$$
\begin{align}
    -x + y &\le 1 \\
    3x + 2y & \le 12 \\
    2x + 3y & \le 12 \\
    x, y & \ge 0 \\
    x, y & \in \mathbb{Z}
\end{align}
$$

The `scipy.optimize.milp()` function can be used to solve this problem. It solves problems of the form: minimize $c^T x$ given:

$$
\begin{align}
    b_l \le Ax \le b_u \\
    l \le x \le u \\
    x_i \in \mathbb{Z} \\
    i \in X_i
\end{align}
$$

where $X_i$ is the set of indicies for which we are optimizing as integers (it can be used for mixed linear programming if desired). The original linear program can be converted so that it is compatible with `milp()`.

In [1]:
import numpy as np
from scipy.optimize import LinearConstraint, milp

Let $x_0 = x$ and $x_1 = y$ from the original problem statment.
Using $x$ as intended for `milp()`, let $x = \begin{pmatrix} x_0 & x_1 \end{pmatrix}$.
Then let $c = \begin{pmatrix} 0 & -1 \end{pmatrix}$ so that $c^T x = -x_1 = -y$. Notice that it is negative since `milp()` solves minimization problems, which can be expresses as negatives of maximization problems.

In [2]:
c = np.array([0, -1])

The matrix $A$ is comprised of the coefficients from the three linear inequalities:

$$
\begin{align}
    -x + y &\le 1 \\
    3x + 2y & \le 12 \\
    2x + 3y & \le 12 \\
\end{align}
$$

and is written as

$$A = \begin{pmatrix}
    -1 & 1 \\
     3 & 2 \\
     2 & 3 
\end{pmatrix}
$$

In [3]:
A = np.array([
    [-1, 1],
    [3, 2],
    [2, 3]
])

The limits of the linear program can be expresses as vectors $b_l$ and $b_u$, called the upper and lower bounds vectors. The values from the original inequalities

$$
\begin{align}
    -x + y &\le 1 \\
    3x + 2y & \le 12 \\
    2x + 3y & \le 12 \\
\end{align}
$$

are used directly for $b_u$. However, there is no lower bound, so $-\infty$ is used for $b_l$:

$$
b_u = \begin{pmatrix} 1 & 12 & 12 \end{pmatrix}
\text{ and }
b_l = \begin{pmatrix} -\infty & -\infty & -\infty \end{pmatrix}
$$.

In [4]:
b_u = np.array([1, 12, 12])
b_l = np.array([-np.inf, -np.inf, -np.inf])

With the problem converted to a form more suitable for `milp()`, we can specify some final constraings before finding the solution.

In [5]:
# The set of linear inequalities are communicated to milp() as a
# LinearConstraint object.
constraints = LinearConstraint(A, b_l, b_u)

# The integrality constraint for a particular variable is indicated
# by passing the value 1 for its position in this array. Other types
# of constraints are available.
integrality = np.array([1, 1])

res = milp(c=c, constraints=constraints, integrality=integrality)

res.x

array([2., 2.])